# Welcome to our main file ^^
## Before you get started, please make sure...
1. You have done generating bitstream in Vivado.
2. Upload `.hwh` and `.bit` file using whatever method to this folder.
3. Upload testing data to be loaded to this folder.

In [1]:
from pynq import Overlay
from pynq import allocate
from pynq import MMIO

## 0. Setting Up

#### Cell Description
This cell is used to set values to be execuded.<br>

Design Files can be found at: <br>
* .bit file:<br>`{ project_location }/{project_name}.runs/impl_1/design_1_wrapper.bit`

* .hwh file:<br>`{ project_location }/{project_name}.gen/{Design Name 1}/bd/{Design Name 2}/hw_handoff/design_1.hwh`<br>

__Note:__ The design files whose filename ends with `.bit` or `.hwh` __must be renamed to be identical__.

In [2]:
# Design File. More detail below.
design_name = "top_warpper.bit"

# Testing Path or Filename
f_test_base_folder = "./Testing_Data/"
f_bias = "bias.txt"
f_ifmaps = "ifmaps.txt"
f_ofmaps = "ofmaps.txt"
f_psum_after_bias = "psum_after_bias.txt"
f_psum_before_bias = "psum_before_bias.txt"
f_settings = "settings.txt"
f_weight = "weight.txt"

# AXI Address
Setting_Kernal_Size_AXI_Address     = 0b1000
Setting_Ofmap_Ifmap_Channel_Address = 0b0000
Setting_Stride_Ofmap_Width_Address  = 0b0100
Write_Weight_Address                = 0b0000
Compute_Start_Address               = 0b0000
Getting_Return_Address              = 0b1100

# Preset Values
IFMAPS_WIDTH = 8;
IFMAPS_HIGHT = 8;
IFMAPS_CH    = 256;
WEIGHT_WIDTH = 3;
WEIGHT_HIGHT = 3;
WEIGHT_NUM   = 1;
STRIDE       = 1;

# Instruction OPCode
INST_WRITE_WEIGHT = 12
INST_COMPUTE      = 87
INST_LOADIFMAPS   = 88

#### Cell Description
This cell is to calcualte the real number (WDATA of AXI) to be transmitted.<br>And read the memory file to be transmitted.

In [9]:
# Calculate WDATAs

# Set Kernal Size
Kernal_Size_WDATA = 0b00001 << (WEIGHT_WIDTH - 1)

# Set Output Feature Map and Input Feature Map Channel Number
Ofmaps_Ifmaps_Channel_WDATA = (WEIGHT_NUM << 12 | IFMAPS_CH) << 8

# Set Stride & Output Feature Map Width
OFMAPS_WIDTH = round(((IFMAPS_WIDTH - WEIGHT_WIDTH) / (STRIDE + 1)));
Stride_Ofmaps_Func_WDATA = OFMAPS_WIDTH << 2 | STRIDE << 12

# ReadMem Function implementation
import numpy as np
def readmem(path, dim):
    with open(f_test_base_folder + path, 'r') as f:
        data = f.read().split()
    data = [int(x, 2) for x in data]
    return np.array(data).reshape(dim)

# Weight memory
weight_dim = (WEIGHT_NUM, IFMAPS_CH , WEIGHT_HIGHT, WEIGHT_WIDTH) ################
weight_mem = readmem(f_weight, weight_dim)
print(weight_mem)

# AXIS Memory
input_buffer = allocate(shape = (1, ), dtype = np.uint32)

ValueError: cannot reshape array of size 189 into shape (1,256,3,3)

#### Cell Description
This cell is used for burning the `.bit` and `.hwh` file onto the PL side.

__Note:__ The filename of these two files __must be identical__.<br>

Only the `.bit` file needs to be added. Overlay constructor will find the `.hwh` file itself

In [ ]:
# Only the .bit file is needed.
ol = Overlay(design_name)

In [ ]:
dma = ol.axi_dma_0
dma_send = ol.axi_dma_0.sendchannel
dma_recv = ol.axi_dma_0.recvchannel

## 1. Sending Command & Preset Values

#### Cell Description

This cell is used to send AXI Lite signal to PL side.<br>
MMIO API Reference: [here](https://pynq.readthedocs.io/en/v3.0.0/pynq_libraries/mmio.html)

In [10]:
mmio = MMIO(0x40000000, 0x1000) # allocate (base address, length)

ret = mmio.read(Getting_Return_Address)
print(ret)

# Set Kernal Size
mmio.write_reg(Setting_Kernal_Size_AXI_Address, Kernal_Size_WDATA)

# Set Output Feature Map and Input Feature Map Channel Number
mmio.write_reg(Setting_Ofmap_Ifmap_Channel_Address, Ofmaps_Ifmaps_Channel_WDATA)

# Set Stride & Output Feature Map Width
mmio.write_reg(Setting_Stride_Ofmap_Width_Address, Stride_Ofmaps_Func_WDATA)

# Send Start Writing Weight
mmio.write_reg(Write_Weight_Address, INST_WRITE_WEIGHT)

# Writing Weight
print('Starting to Write Weight')
for x in range(WEIGHT_NUM):
    for y in range(WEIGHT_WIDTH):
        for z in range(0, WEIGHT_WIDTH, 6):
            
            input_buffer[0] = 0
            
            for w in range(0, 30):
                if not w % 5 >= WEIGHT_HIGHT or (w / 5 + z) >= IFMAPS_CH:
                    input_buffer[0] |= weight_mem[x][round(w / 5) + z][w % 5][y] << (w - 1)
            
            dma_send.transfer(input_buffer)
            # dma_send.wait()
            print("Input = {}".format(hex(input_buffer[0])))

# Read Return Value
ret = 0
print('\nret = {}. Getting return value.'.format(ret), end = '')
while ret != 0xFFFFFFFF:
    ret = mmio.read(Getting_Return_Address)
    print('.', end = '')
print('\nret = {}. Getting return value done.'.format(ret))

0
Starting to Write Weight


NameError: name 'weight_mem' is not defined

## 2. Start Compute
### Work In Progress..

#### Cell Description

Set up DMA

In [ ]:
mmio.read(0)